# Cheap Optical Flow: Is It Good?


## Quickstart

## Credits

Some portions of this notebook adapted from:
 * [Middlebury Flow code by Johannes Oswald](https://github.com/Johswald/flow-code-python/blob/master/readFlowFile.py)
 * [DeepDeform Demo Code](https://github.com/AljazBozic/DeepDeform)
 * [OpticalFlowToolkit by RUOTENG LI](https://github.com/liruoteng/OpticalFlowToolkit)
 * [OpenCV Samples](https://github.com/opencv/opencv/blob/master/samples/python/opt_flow.py)

In [ ]:
# parameters
SHOW_DEMO_OUTPUT = True

In [ ]:
## Data Model & Utility Code

!pip3 install pypng

import attr
import cv2
import imageio
import IPython.display
import os
import PIL.Image
import six

from oarphpy import plotting as op_plt
import numpy as np


@attr.s(slots=True, eq=False, weakref_slot=False)
class OpticalFlowPair(object):
    """A flyweight for a pair of images with an optical flow field"""
    
    dataset = attr.ib(type=str, default='')
    """To which dataset does this pair belong?"""
    
    id1 = attr.ib(type=str, default='')
    """Identifier or URI for the first image"""
    
    id2 = attr.ib(type=str, default='')
    """Identifier or URI for the second image"""
    
    img1 = attr.ib(default=None)
    """URI or numpy array for the first image (source image)"""

    img2 = attr.ib(default=None)
    """URI or numpy array for the second image (target image)"""
    
    flow = attr.ib(default=None)
    """A callable or numpy array representing optical flow from img1 -> img2"""
    
    def to_html(self):
        im1 = load_image(self.img1)
        im2 = load_image(self.img2)
        flow = self.flow if isinstance(self.flow, (np.ndarray, np.generic)) else self.flow()
        fviz = draw_flow(im1, flow)
        html = """
            <table>
            
            <tr><td style="text-align:left"><b>Dataset:</b> {dataset}</td></tr>
            
            <tr><td style="text-align:left"><b>Source Image:</b> {id1}</td></tr>
            <tr><td><img src="{im1}" width="100%" /></td></tr>

            <tr><td style="text-align:left"><b>Target Image:</b> {id2}</td></tr>
            <tr><td><img src="{im2}" width="100%" /></td></tr>

            <tr><td style="text-align:left"><b>Flow</b></td></tr>
            <tr><td><img src="{fviz}" width="100%" /></td></tr>
            </table>
        """.format(
                dataset=self.dataset,
                id1=self.id1, id2=self.id2,
                im1=img_to_data_uri(im1), im2=img_to_data_uri(im2),
                fviz=img_to_data_uri(fviz))
        return html


## General Utilities
    
def imshow(x):
    IPython.display.display(PIL.Image.fromarray(x))

def show_html(x):
    from IPython.core.display import display, HTML
    display(HTML(x))
    
def load_image(x):
    if isinstance(x, (np.ndarray, np.generic)):
        return x
    if isinstance(x, six.string_types):
        return imageio.imread(x)
    else:
        raise ValueError("Can't handle %s" % (x,))

img_to_data_uri = lambda x: op_plt.img_to_data_uri(x, format='png')
# # TODO correct oarphpy img uri to be png or jpeg   data_url = 'data:image/png;base64,{}'.format(parse.quote(data)) 
# def img_to_data_uri(img, format='png', jpeg_quality=75):
#     """Given a numpy array `img`, return a `data:` URI suitable for use in 
#     an HTML image tag."""

#     from io import BytesIO
#     out = BytesIO()

#     import imageio
#     kwargs = dict(format=format)
#     if format == 'jpg':
#         kwargs.update(quality=jpeg_quality)
#     imageio.imwrite(out, img, **kwargs)

#     from base64 import b64encode
#     data = b64encode(out.getvalue()).decode('ascii')

#     from six.moves.urllib import parse
#     data_url = 'data:image/png;base64,{}'.format(parse.quote(data))

#     return data_url


def draw_flow(img, flow, step=8):
    """Based upon OpenCV sample: https://github.com/opencv/opencv/blob/master/samples/python/opt_flow.py"""
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = img.copy()
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis


## Middlebury Optical Flow



In [ ]:
# Please unzip `other-color-allframes.zip` and `other-gt-flow.zip` to a directory and provide the target below:
MIDD_DATA_ROOT = '/outer_root/host_mnt/Volumes/970-evo-raid0/middlebury-flow/'

# For the Middlebury Flow dataset, we only consider the real scenes
MIDD_SCENES = [
    {
        'input': 'other-data/Dimetrodon/frame10.png',
        'expected_out': 'other-data/Dimetrodon/frame11.png',
        'flow_gt': 'other-gt-flow/Dimetrodon/flow10.flo',
    },
        {
        'input': 'other-data/Hydrangea/frame10.png',
        'expected_out': 'other-data/Hydrangea/frame11.png',
        'flow_gt': 'other-gt-flow/Hydrangea/flow10.flo',
    },
        {
        'input': 'other-data/RubberWhale/frame10.png',
        'expected_out': 'other-data/RubberWhale/frame11.png',
        'flow_gt': 'other-gt-flow/RubberWhale/flow10.flo',
    },
]


def midd_read_flow(file):
    # Based upon: https://github.com/Johswald/flow-code-python/blob/master/readFlowFile.py
    # compute colored image to visualize optical flow file .flo
    # Author: Johannes Oswald, Technical University Munich
    # Contact: johannes.oswald@tum.de
    # Date: 26/04/2017
    # For more information, check http://vision.middlebury.edu/flow/ 

    assert type(file) is str, "file is not str %r" % str(file)
    assert os.path.isfile(file) is True, "file does not exist %r" % str(file)
    assert file[-4:] == '.flo', "file ending is not .flo %r" % file[-4:]
    f = open(file, 'rb')
    flo_number = np.fromfile(f, np.float32, count=1)[0]
    TAG_FLOAT = 202021.25
    assert flo_number == TAG_FLOAT, 'Flow number %r incorrect. Invalid .flo file' % flo_number
    w = np.fromfile(f, np.int32, count=1)
    h = np.fromfile(f, np.int32, count=1)

    #if error try: data = np.fromfile(f, np.float32, count=2*w[0]*h[0])
    data = np.fromfile(f, np.float32, count=int(2*w*h))
    
    # Reshape data into 3D array (columns, rows, bands)
    flow = np.resize(data, (int(h), int(w), 2))	
    f.close()
    
    # We found that there are some invalid (?) (i.e. very large) flows, so we're going
    # to ignore those for this experiment.
    invalid = (flow >= 1666)
    flow[invalid] = 0

    return flow


for i, scene in enumerate(MIDD_SCENES):
    p = OpticalFlowPair(
            dataset="Middlebury Optical Flow",
            id1=scene['input'],
            img1='file://' + os.path.join(MIDD_DATA_ROOT, scene['input']),
            id2=scene['expected_out'],
            img2='file://' + os.path.join(MIDD_DATA_ROOT, scene['expected_out']),
            flow=lambda: midd_read_flow(os.path.join(MIDD_DATA_ROOT, scene['flow_gt'])))
    
    if SHOW_DEMO_OUTPUT:
        show_html(p.to_html() + "<br/><br/><br/>")


## DeepDeform

In [ ]:
# Please extract deepdeform_v1.7z to a directory and provide the target below:
DD_DATA_ROOT = '/outer_root/host_mnt/Volumes/970-evo-raid0/deepdeform_v1/'

DD_DEMO_SCENES = [
    {
        "input": "train/seq000/color/000000.jpg",
        "expected_out": "train/seq000/color/000200.jpg",
        "flow_gt": "train/seq000/optical_flow/blackdog_000000_000200.oflow",
    },
    
    {
        "input": "train/seq000/color/000000.jpg",
        "expected_out": "train/seq000/color/001200.jpg",
        "flow_gt": "train/seq000/optical_flow/blackdog_000000_001200.oflow",
    },
    
    {
        "input": "train/seq001/color/003400.jpg",
        "expected_out": "train/seq001/color/003600.jpg",
        "flow_gt": "train/seq001/optical_flow/lady_003400_003600.oflow",
    },
    
    {
        "input": "train/seq337/color/000050.jpg",
        "expected_out": "train/seq337/color/000350.jpg",
        "flow_gt": "train/seq337/optical_flow/adult_000050_000350.oflow",
    },
]

def dd_load_flow(path):
    # Based upon https://github.com/AljazBozic/DeepDeform/blob/master/utils.py#L1
    import shutil
    import struct
    
    # Flow is stored row-wise in order [channels, height, width].
    assert os.path.isfile(path)

    flow_gt = None
    with open(path, 'rb') as fin:
        width = struct.unpack('I', fin.read(4))[0]
        height = struct.unpack('I', fin.read(4))[0]
        channels = struct.unpack('I', fin.read(4))[0]
        n_elems = height * width * channels

        flow = struct.unpack('f' * n_elems, fin.read(n_elems * 4))
        flow_gt = np.asarray(flow, dtype=np.float32).reshape([channels, height, width])
    
    # Match format used in this analysis
    flow_gt = np.moveaxis(flow_gt, 0, -1) # (h, w, 2)
    invalid_flow = flow_gt == -np.Inf
    flow_gt[invalid_flow] = 0.0
    return flow_gt

def dd_create_fp(info):
     return OpticalFlowPair(
                dataset="DeepDeform Semi-Synthetic Optical Flow",
                id1=scene['input'],
                img1='file://' + os.path.join(DD_DATA_ROOT, scene['input']),
                id2=scene['expected_out'],
                img2='file://' + os.path.join(DD_DATA_ROOT, scene['expected_out']),
                flow=lambda: dd_load_flow(os.path.join(DD_DATA_ROOT, scene['flow_gt'])))

import json
DD_ALIGNMENTS = json.load(open(os.path.join(DD_DATA_ROOT, 'train_alignments.json')))
ALL_DD_SCENES = [
    {
        "input": ascene['source_color'],
        "expected_out": ascene['target_color'],
        "flow_gt": ascene['optical_flow'],
    }
    for ascene in DD_ALIGNMENTS
]

print("Found %s DeepDeform scenes" % len(ALL_DD_SCENES))
if SHOW_DEMO_OUTPUT:
    for scene in DD_DEMO_SCENES:
        p = dd_create_fp(scene)
        show_html(p.to_html())
    
# for i, scene in enumerate(ALL_DD_SCENES):
    

    
#     img_in = imageio.imread(os.path.join(DD_DATA_ROOT, scene['input']))
#     expected = imageio.imread(os.path.join(DD_DATA_ROOT, scene['expected_out']))
#     flow_gt = dd_load_flow(os.path.join(DD_DATA_ROOT, scene['flow_gt']))
    
#     show_scene(img_in, expected, flow_gt, dataset='deepdeform', exname=str(i))
#     continue
    
#     vis = draw_flow(img_in, flow_gt)
#     imshow(vis)

#     warped = warp_flow(img_in, flow_gt[:, :, :2])
#     imshow(warped)
    
#     exclude = (flow_gt == np.array([0, 0])).all(axis=-1)
#     warped[exclude] = np.array([0, 0, 0])
#     imshow(warped)
    
#     imshow(expected)



In [ ]:
## Kitti Scene Flow Benchmark (2015)


In [ ]:
# Please unzip `data_scene_flow.zip` and `data_scene_flow_calib.zip` to a directory and provide that target below:
KITTI_SF15_DATA_ROOT = '/outer_root/host_mnt/Volumes/970-evo-raid0/kitti_sceneflow_scratch/'

# You have to ls flow_occ 
KITTI_SF15_DEMO_SCENES = [
    {
        'input': 'training/image_2/000000_10.png',
        'expected_out': 'training/image_2/000000_11.png',
        'flow_gt': 'training/flow_occ/000000_10.png',
    },
    {
        'input': 'training/image_2/000007_10.png',
        'expected_out': 'training/image_2/000007_11.png',
        'flow_gt': 'training/flow_occ/000007_10.png',
    },
    {
        'input': 'training/image_2/000023_10.png',
        'expected_out': 'training/image_2/000023_11.png',
        'flow_gt': 'training/flow_occ/000023_10.png',
    },
    {
        'input': 'training/image_2/000051_10.png',
        'expected_out': 'training/image_2/000051_11.png',
        'flow_gt': 'training/flow_occ/000051_10.png',
    },
    {
        'input': 'training/image_2/000003_10.png',
        'expected_out': 'training/image_2/000003_11.png',
        'flow_gt': 'training/flow_occ/000003_10.png',
    },
]

from oarphpy import util as oputil
KITTI_SF15_ALL_FLOW_OCC = [
    os.path.basename(p)
    for p in oputil.all_files_recursive(
        os.path.join(KITTI_SF15_DATA_ROOT, 'training/flow_occ'), pattern='*.png')
]
    
KITTI_SF15_ALL_SCENES = [
    {
        "input": 'training/image_2/%s' % fname,
        "expected_out": 'training/image_2/%s' % fname.replace('_10', '_11'),
        "flow_gt": 'training/flow_occ/%s' % fname,
    }
    for fname in KITTI_SF15_ALL_FLOW_OCC
]
print("Found %s KITTI SceneFlow 2015 scenes" % len(KITTI_SF15_ALL_SCENES))


def kitti_sf15_load_flow_from_png(path):
    # Based upon https://github.com/liruoteng/OpticalFlowToolkit/blob/master/lib/flowlib.py#L559
    import png
    flow_object = png.Reader(filename=path)
    flow_direct = flow_object.asDirect()
    flow_data = list(flow_direct[2])
    w, h = flow_direct[3]['size']
    flow = np.zeros((h, w, 3), dtype=np.float64)
    for i in range(len(flow_data)):
        flow[i, :, 0] = flow_data[i][0::3]
        flow[i, :, 1] = flow_data[i][1::3]
        flow[i, :, 2] = flow_data[i][2::3]

    invalid_idx = (flow[:, :, 2] == 0)
    flow[:, :, 0:2] = (flow[:, :, 0:2] - 2 ** 15) / 64.0
    flow[invalid_idx, 0] = 0
    flow[invalid_idx, 1] = 0
    return flow[:, :, :2]

def kitti_sf15_create_fp(info):
     return OpticalFlowPair(
                dataset="KITTI Scene Flow 2015",
                id1=scene['input'],
                img1='file://' + os.path.join(KITTI_SF15_DATA_ROOT, scene['input']),
                id2=scene['expected_out'],
                img2='file://' + os.path.join(KITTI_SF15_DATA_ROOT, scene['expected_out']),
                flow=lambda: kitti_sf15_load_flow_from_png(os.path.join(KITTI_SF15_DATA_ROOT, scene['flow_gt'])))

if SHOW_DEMO_OUTPUT:
    for scene in KITTI_SF15_DEMO_SCENES:
        p = kitti_sf15_create_fp(scene)
        show_html(p.to_html())